In [1]:
# Imports
import gradio as gr
import g4f
import tiktoken

In [11]:
# Count tokens
system_message = [{"role": "system", "content": "Ты пират, отвечай как пират."}]
max_response_tokens = 1024
token_limit = 16000
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [3]:
def gradio_to_openai_history(history):
    result = []
    for dialogue in history:
        result.append({"role": "user", "content": dialogue[0]})
        result.append({"role": "assistant", "content": dialogue[1]})
    return result

In [9]:
def response(message, history):
    conversation = system_message + gradio_to_openai_history(history) + [{"role": "user", "content": message}]
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
    return g4f.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        provider=g4f.Provider.NeuroGPT,
        messages=conversation,
        stream=False,
        )

demo = gr.ChatInterface(response)

In [12]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
